In [2]:
#THaW Project
#This program is meant to make a Naive Bayes algorithm to test the data collected by MSU
#6/12/2019
#Code inspired by https://xavierbourretsicotte.github.io/Naive_Bayes_Classifier.html


import pandas as pd
import os
import csv
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
import itertools
from scipy.stats import norm
import scipy.stats
from sklearn.naive_bayes import GaussianNB


#Load the dataset
path=r'C:\Users\brnma\output.xlsx'
df = pd.ExcelFile(path).parse('Sheet1')
df = df.rename(index = str, columns = {'AmplitudeKurtosis':'1_AmplitudeKurtosis', 'AmplitudeMean':'2_AmplitudeMean',
                                       'AmplitudeSkew':'3_AmplitudeSkew','AmplitudeStdDev':'4_AmplitudeStdDev',
                                       'AmplitudeVariance':'5_AmplitudeVariance','PhaseKurtosis':'6_PhaseKurtosis',
                                       'PhaseMean':'7_PhaseMean','PhaseSkew':'8_PhaseSkew','PhaseStdDev':'9_PhaseStdDev'
                                       ,'PhaseVariance':'10_PhaseVariance'})

df = df[~df['Device'].isin([' Samsung Hub1', 'Iris Hub', 'Sengled Bulb 2'])]
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)#Remove unnecessary column

#Plot the scatter of Amplitude Mean vs Phase Mean
sns.FacetGrid(df, hue="Device", height=20) .map(plt.scatter,"2_AmplitudeMean", "7_PhaseMean", )  .add_legend()
plt.title('Scatter plot')
df1 = df[["2_AmplitudeMean", "7_PhaseMean",'Device']]

def predict_NB_gaussian_class(X,mu_list,std_list,pi_list): 
    #Returns the class for which the Gaussian Naive Bayes objective function has greatest value
    scores_list = []
    classes = len(mu_list)
    
    for p in range(classes):
        score = (norm.pdf(x = X[0], loc = mu_list[p][0][0], scale = std_list[p][0][0] )  
                * norm.pdf(x = X[1], loc = mu_list[p][0][1], scale = std_list[p][0][1] ) 
                * pi_list[p])
        scores_list.append(score)
             
    return np.argmax(scores_list)

def predict_Bayes_class(X,mu_list,sigma_list): 
    #Returns the predicted class from an optimal bayes classifier - distributions must be known
    scores_list = []
    classes = len(mu_list)
    
    for p in range(classes):
        score = scipy.stats.multivariate_normal.pdf(X, mean=mu_list[p], cov=sigma_list[p])
        scores_list.append(score)
             
    return np.argmax(scores_list)

#Estimating the parameters
mu_list = np.split(df1.groupby('Device').mean().values,[1,2])
std_list = np.split(df1.groupby('Device').std().values,[1,2], axis = 0)
pi_list = df1.iloc[:,2].value_counts().values / len(df1)

# Our 2-dimensional distribution will be over variables X and Y
N = 100
X = np.linspace(-0.5, 0.5, N)
Y = np.linspace(-0.5, 0.5, N)
X, Y = np.meshgrid(X, Y)

#fig = plt.figure(figsize = (10,10))
#ax = fig.gca()
color_list = ['Blues','Greens','Reds']
my_norm = colors.Normalize(vmin=-1.,vmax=1.)

g = sns.FacetGrid(df, hue="Device", height=10, palette = 'colorblind') .map(plt.scatter, "2_AmplitudeMean", "7_PhaseMean",)  .add_legend()
my_ax = g.ax


#Computing the predicted class function for each value on the grid
zz = np.array(  [predict_NB_gaussian_class( np.array([xx,yy]).reshape(-1,1), mu_list, std_list, pi_list) 
                     for xx, yy in zip(np.ravel(X), np.ravel(Y)) ] )


#Reshaping the predicted class into the meshgrid shape
Z = zz.reshape(X.shape)


#Plot the filled and boundary contours
my_ax.contourf( X, Y, Z, 2, alpha = .1, colors = ('blue','green','red'))
my_ax.contour( X, Y, Z, 2, alpha = 1, colors = ('blue','green','red'))

# Addd axis and title
my_ax.set_xlabel('Amplitude Mean')
my_ax.set_ylabel('Phase Mean')
my_ax.set_title('Gaussian Naive Bayes decision boundaries')

plt.show()


#NEW BLOCK

# from sklearn.naive_bayes import GaussianNB

# #Setup X and y data
# X_data = df1.iloc[:,0:2]
# [int(i) for i in X_data]
# y_labels = int(df1.iloc[:,2].replace({' Samsung Hub2':0,'Sengled Bulb 1':1,'Sylvania Bulb':2}).copy())


# #Fit model
# model_sk = GaussianNB(priors = None)
# model_sk.fit(X_data,y_labels)


# # Our 2-dimensional classifier will be over variables X and Y
# N = 100
# X = np.linspace(4, 8, N)
# Y = np.linspace(1.5, 5, N)
# X, Y = np.meshgrid(X, Y)

# #fig = plt.figure(figsize = (10,10))
# #ax = fig.gca()
# color_list = ['Blues','Greens','Reds']
# my_norm = colors.Normalize(vmin=-1.,vmax=1.)

# g = sns.FacetGrid(iris, hue="Device", size=10, palette = 'colorblind') .map(plt.scatter, "2_AmplitudeMean", "7_PhaseMean",)  .add_legend()
# my_ax = g.ax


# #Computing the predicted class function for each value on the grid
# zz = np.array(  [model_sk.predict( [[xx,yy]])[0] for xx, yy in zip(np.ravel(X), np.ravel(Y)) ] )


# #Reshaping the predicted class into the meshgrid shape
# Z = zz.reshape(X.shape)


# #Plot the filled and boundary contours
# my_ax.contourf( X, Y, Z, 2, alpha = .1, colors = ('blue','green','red'))
# my_ax.contour( X, Y, Z, 2, alpha = 1, colors = ('blue','green','red'))

# # Addd axis and title
# my_ax.set_xlabel('Amplitude Mean')
# my_ax.set_ylabel('Phase Mean')
# my_ax.set_title('Gaussian Naive Bayes decision boundaries')

# plt.show()

# #Numpy accuracy
# y_pred = np.array(  [predict_NB_gaussian_class( np.array([xx,yy]).reshape(-1,1), mu_list, std_list, pi_list) 
#                      for xx, yy in zip(np.ravel(X_data.values[:,0]), np.ravel(X_data.values[:,1])) ] )
# display(np.mean(y_pred == y_labels))

# #Sklearn accuracy
# display(model_sk.score(X_data,y_labels))

# print("HHHEHHEHHE")
# For all devices, probability that we have seen said device is 1/numDevices
# For all features, probability that feauture is some feature is 

NameError: name 'ninetyTenSplit' is not defined

In [ ]:
 def meanDevices(df):
#     global devices
#     #Should be a table showing the mean values for all of the devices and their respective features:
    
#     numDevices = len(devices)
#     means = pd.DataFrame(index = devices, columns=df.columns[3:])
    
#     for feat in  means.columns:
#         i = 0
#         count = 0
#         for device in means.index:
#             total = 0
#             count = 0
#             while i <= df.index.max() and df['Device'][i] == device:
#                 total+=float(df[feat][i])
#                 i+=1    
#                 count+=1
#             means[feat][device] = total/count
#     print(means)
#     return means
# meanDevices(df)

In [ ]:
def meanAndSD(df):
    global devices
    
    numDevices = len(devices)
    stdevs = pd.DataFrame(index = devices, columns=df.columns[3:])
    means = pd.DataFrame(index = devices, columns=df.columns[3:])
    
    for feat in stdevs.columns:
        i = 0
        stdevList = []
        meansList = []
        
        if i not in df.index:
            i+=1
        device = df['Device'][i]
        while i <= df.index.max():
            
            if df['Device'][i] == device:
                stdevList.append(df[feat][i])
                meansList.append(df[feat][i])
                i+=1
                if i not in df.index and i < df.index.max():
                    i+=1
            else:  
               
                stdevs[feat][device] = statistics.stdev(stdevList)
                means[feat][device] = statistics.mean(meansList)
                
                stdevList = []
                meansList = []
                device = df['Device'][i]
    
        stdevs[feat][device] = statistics.stdev(stdevList)
        means[feat][device] = statistics.mean(meansList)
        
    stats = {}
    stats['means'] = means
    stats['stdevs'] = stdevs
    
    return stats

# meanAndSD(df)  
